# Connecting LLM with Agent on SQL Databases (using mostly google cloud)

Information for new Colab users before running the project: \
1. Colab uses Virtual Machines (VM) to run Python Scripts. You can activate one by pressing the Connect button right top corner. It is currently set on using Standard CPU and GPU and High Ram, but you can customize it if you want.
2. In Colab you can Collapse and Extend cells to improve the viewablity. The next block contains 2 code cells of downloading the libraries. You can simply press the play icon if you hover over the "2 cells hidden" text to automatically run the code after one another without looking at the code.

Information about the project beforehand: \
1. After the Libraries cell block, you have the LLM Retrieval cell block. This block lets you choose whether you prefer to activate Gemini's (vertex_llm) and/or OpenAI's model (OpenAIllm). You need to read this code part a bit yourself to understand it properly. Like Gemini needs your account info to access Vertex models, while OpenAI uses a simple API key.
2. After the LLM Retrieval cell block, you have the Database Connection cell block. This extracts the database files out of (in this case) Google cloud. Each database is stored together in a single variable using function "extract_databases_from_gcs". You can use either Bird-bench or Spider to benchmark the LLMs. (You can only use one to place in the variable "test_databases")
3. After the Database Connection cell block, you have the Benchmarking cell block. Here you can configure the parameters for both the agents and the LLM (be careful to use correct wording for each benchmark). Afterwards each agent is configured (no need to change anything there if you don't want to) and some extra functions to properly deal with some data structures like in the "Loading function" cell. After that the "running" cell shows live how the process is ran. If you want to see what each agent thinks, you can turn on the "verbose" parameter in the configure agent function. Currently only the thinking process of the manager agent is turned on.
4. During the running process of the "running" cell, after each database is finished its output is stored in the folder that is indicated by the variable "output_folder" (see cell after "Benchmarking" cell block) like "bird-bench/benchmarks/senior_geminiv5/california_schools.json". Currently this is stored within Google Cloud Console -> Buckets -> Dashboardllmbucket -> "output folder".
5. The naming of the google cloud folders are based off the parameters. A name like "refiner_geminiv3(5)" uses All agents + Gemini + 3 Query amount + 5 Few shots. A name like "senior_geminiv5" uses Manager, Database and Senior agent + Gemini + 5 Query Amount.


## Libraries

In [ ]:
!pip install pydantic==2.9.2
!pip install transformers==4.41.2
!pip install langchain-openai
!pip install langchain-community
!pip install langchain_experimental
!pip install -U langchain-google-genai
!pip install langchain_google_vertexai
!pip install google-cloud-language>=2.9.1
!pip install psycopg2
!sudo apt-get update && sudo apt-get install google-cloud-sdk
!pip install --upgrade google-genai
!pip install -U google-cloud-aiplatform "shapely<2"

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,943 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,467 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,151 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Pac

In [ ]:
# =======================
# Standard Library Imports
# =======================
import ast
import base64
import getpass
import io
import json
import os
import random
import re
import sqlite3
import tempfile
import time
import uuid
import warnings
from collections import defaultdict
from operator import itemgetter
from sqlite3 import OperationalError
from typing import (
    TYPE_CHECKING,
    Any,
    Annotated,
    Dict,
    List,
    Literal,
    Optional,
    Sequence,
    Union,
    cast,
)

# =======================
# Suppress Warnings
# =======================
from sqlalchemy.exc import SAWarning
warnings.filterwarnings(
    "ignore",
    message=".*Cannot correctly sort tables; there are unresolvable cycles.*",
    category=SAWarning,
)

# =======================
# Third-Party Library Imports
# =======================
import pandas as pd
import psycopg2
import sentencepiece
import sqlparse
import torch
from IPython.display import Image, display
from pydantic import BaseModel, Field
from sqlparse.sql import TokenList

# =======================
# Google Cloud & VertexAI
# =======================
from google import genai
from google.cloud import storage
from google.colab import auth
from google.genai import types
import vertexai
from langchain_google_vertexai import ChatVertexAI

# =======================
# HuggingFace Transformers
# =======================
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Conversation,
    LlamaForCausalLM,
    LlamaTokenizer,
    RobertaForSequenceClassification,
    RobertaTokenizer,
    pipeline,
)

# =======================
# LangChain Core
# =======================
from langchain.agents import AgentExecutor, AgentType, create_tool_calling_agent, tool
from langchain.agents.agent import (
    RunnableAgent,
    RunnableMultiActionAgent,
)
from langchain.chains import create_sql_query_chain
from langchain.llms.base import BaseLLM
from langchain.memory import ChatMessageHistory
from langchain.schema import AIMessage, HumanMessage
from langchain.sql_database import SQLDatabase
from langchain.tools.base import Tool

# LangChain Core Modules
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage as CoreAIMessage, SystemMessage, ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_core.runnables import (
    RunnableLambda,
    RunnablePassthrough,
    RunnableWithFallbacks,
    chain,
)
from langchain_core.runnables.graph import MermaidDrawMethod
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tools import tool as core_tool

# =======================
# LangChain Community
# =======================
from langchain_community.agent_toolkits import SQLDatabaseToolkit, create_sql_agent
from langchain_community.agent_toolkits.sql.prompt import (
    SQL_FUNCTIONS_SUFFIX,
    SQL_PREFIX,
    SQL_SUFFIX,
)
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit as CommunitySQLToolkit
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
)
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

from langchain_community.utilities import SQLDatabase as CommunitySQLDatabase
from langchain_openai import ChatOpenAI
# =======================
# Typing Extensions
# =======================
from typing_extensions import TypedDict
from typing import Union

## LLM Retrieval

### Vertex AI models
gemini-2.0-flash-001


In [ ]:
'''Option 1: using google authentication (requires colab user to have access with google cloud project for gemini)'''
auth.authenticate_user()

'''Option 2: using a key file'''
# from google.colab import files
# uploaded = files.upload()
# key_path = list(uploaded.keys())[0] #gets the uploaded file name.
# print(key_path)
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path

print('''Authentication done!''')

Authentication done!


In [ ]:
PROJECT_ID = "dashboard-llm-454014"  # Replace with your project ID
REGION = "europe-west1"  # Use the region where your model is available.

# Initialize Vertex AI (using default credentials)
vertexai.init(project=PROJECT_ID, location=REGION)

vertex_llm = ChatVertexAI(
    model="gemini-2.0-flash-001", #could be any model, preferably gemini-2.0-flash-001 for good and cheapest results
    temperature=0,
    max_tokens=2048,
    max_retries=16,
    stop=None,
    location=REGION,
)

print('''Initialized Gemini model as "vertex_llm"''')

Initialized Gemini model as "vertex_llm"


In [ ]:
# def custom_token_encoder(text: str) -> List[int]:
#     """Custom function to encode text into token IDs."""
#     return [ord(char) for char in text]  # Example encoding (replace with actual tokenizer)

# def count_tokens(text: str) -> int:
#     """Counts the number of tokens in a given text."""
#     return len(custom_token_encoder(text))  # Use your tokenizer here

### OpenAI models
"gpt-4o", "gpt-4", "gpt-4o-mini"

In [ ]:
openai_key = ""

#### Access

In [ ]:
def _set_env(key: str, value: str):
    if key not in os.environ:
        os.environ[key] = value

_set_env("OPENAI_API_KEY", openai_key)

#### Retrieval

In [ ]:
OpenAIllm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_key)

In [ ]:
'''Optionally testing the LLM'''
# OpenAIllm.invoke("Hi, what is your name?")

'Optionally testing the LLM'

## Database connection
This Demo uses data from buckets within the google cloud console (=gcs).\
If you will only use ChatGPT, I would advice using google drive to integrate the data within the project instead of google cloud console

In [ ]:
def extract_databases_from_gcs(bucket_name, dataset_folder):
    databases = {}
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    blobs = client.list_blobs(bucket_name, prefix=dataset_folder)

    for blob in blobs:

        file_name = blob.name.split("/")[-1] #get filename
        folder_name = blob.name.split("/")[-2] #get folder name

        if file_name.endswith(".DS_Store"):
            continue
        elif file_name.endswith(".json"):
            continue
        elif file_name.endswith(".sqlite"): #ensure the sqlite file is the correct one.
            # Download the SQLite file content into memory
            sqlite_data = blob.download_as_bytes()

            # Create a temporary file
            with tempfile.NamedTemporaryFile(delete=False, suffix=".sqlite") as temp_sqlite_file:
                temp_sqlite_file.write(sqlite_data)
                temp_file_path = temp_sqlite_file.name

            # Create a URI for the temporary SQLite database
            db_uri = f"sqlite:///{temp_file_path}"

            # Create the SQLDatabase object
            db = SQLDatabase.from_uri(db_uri)

            databases[folder_name] = db

    return databases

### Bird-Bench Database

In [ ]:
bucket_name = "dashboardllmbucket"  # Replace with your bucket name
test_dataset_folder = "bird-bench/dev/dev_databases"

test_databases = extract_databases_from_gcs(bucket_name, test_dataset_folder)

test_databases_names = list(test_databases.keys())

print(test_databases_names) #the code is build that it loops through this list of database names

['california_schools', 'card_games', 'codebase_community', 'debit_card_specializing', 'european_football_2', 'financial', 'formula_1', 'student_club', 'superhero', 'thrombosis_prediction', 'toxicology']


### Spider

In [ ]:
# bucket_name = "dashboardllmbucket"  # Replace with your bucket name
# test_dataset_folder = "spider/dev/database"

# test_databases = extract_databases_from_gcs(bucket_name, test_dataset_folder)

# test_databases_names = list(test_databases.keys())

# print(test_databases_names) #the code is build that it loops through this list of database names

## Benchmarking

In [ ]:
# Change which agent should have which LLM as a core. (Potentially you can make them all gemini/vertex or all gpt)
databasellm = vertex_llm
refinerllm = vertex_llm
seniorllm = vertex_llm
managerllm = vertex_llm

# Output path for JSON files (currently is also google cloud)
bucket_name = "dashboardllmbucket"

output_folder = "bird-bench/benchmarks/senior_geminiv5" #bird-bench or spider (the folder is automatically created if it doesn't exist)
question_path = "bird-bench/dev/dev.json" #bird-bench or spider
few_shot_path = "bird-bench/dev/dev.json" #bird-bench or spider

# output_folder = "spider/benchmarks/senior_geminiv5" #bird-bench or spider (the folder is automatically created if it doesn't exist)
# question_path = "spider/dev/dev.json" #bird-bench or spider
# few_shot_path = "spider/dev/dev.json" #bird-bench or spider

few_shot_amount = 10
query_table_name = "SQL" #JSON table name where the correct SQL is stored (SQL for bird, query for spider)
query_amount = 5

# All optional agents to potentially improve the usage of the database llm agent
use_manager_agent = True #helps organize all the agents and improve the semantic understanding of the question
use_senior_agent = True #helps indicate which columns/tables it should use (requires manager agent to be True)
use_refiner_agent = True #helps refine the queries if all are incorrect (requires manager agent to be True)

#### Configure Database agent

In [ ]:

def configure_databasellmv2(database_name, bucket_name, few_shot_path, few_shot_amount=3, query_amount=1, query_table_name='query'):
  sql_query_store = []
  toolkit = SQLDatabaseToolkit(db=db, llm=databasellm)
  list_sql_database_tool = toolkit.get_tools()[2]

  info_sql_database_tool = toolkit.get_tools()[1]
  info_sql_database_tool.description = (
        "Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. "
        f"Be sure that the tables actually exist by calling {list_sql_database_tool.name} first! "
        "Example Input: table1, table2, table3"
    )

  query_sql_database_tool = toolkit.get_tools()[0]
  query_sql_database_tool.description= (
              "Input to this tool is a detailed and correct SQL query, output is a "
              "result from the database. If the query is not correct, an error message "
              "will be returned. If an error is returned, rewrite the query, check the "
              "query, and try again. If you encounter an issue with Unknown column "
              f"'xxxx' in 'field list', use {info_sql_database_tool.name} "
              "to query the correct table fields."
          )

  databasellm_tools = [
      list_sql_database_tool,
      info_sql_database_tool,
      query_sql_database_tool,
  ]

  # Load training data from GCS
  training_data = load_json_from_gcs(bucket_name, few_shot_path)

  # Creating prompt given parameters
  database_train = [entry for entry in training_data if entry.get("db_id") == database_name]
  prompt = f"""You are an AI SQL agent specializing in generating and executing **syntactically correct SQL queries** in SQLite language."""

  #Determining in prompt how many queries it should make
  if query_amount == 1:
    prompt += f"""
    Return **only** a single final SQL query—without explanations, reasoning, or extra text—that can be valid answers to the prompt."""
  else:
    prompt += f"""
    Return **only** {query_amount} different final SQL queries—without explanations, reasoning, or extra text—that can be valid answers to the prompt."""

  if few_shot_amount != 0:
    few_shot_examples = "\n".join(
        [f"Question: {ex['question']}\nSQL: {ex[query_table_name]}" for ex in database_train[:few_shot_amount]]
    )
    prompt += f"""
  ### **Database Knowledge**
  Examples of correct queries based on the database structure:
    {few_shot_examples}"""

  prompt += f"""
  ### **SQL Generation Rules**"""
  #Determining in prompt how many queries it should make
  if query_amount == 1:
    prompt += f"""
    - **Only return a single SQL query—no explanations.**
    """
  else:
    prompt += f"""
  - **Only return {query_amount} SQL queries—no explanations.**
  - **Each query MUST use a different primary table in the SELECT clause if the Table usage is the same.**
  - Also fluctuate the where clause generation of each query, by using "LIKE '%word%'" instead of "= 'thisisalongword'.
  - Also fluctuate the generation of each query, for between tables and without or without GROUP BY
  - Use **different aggregation methods** (COUNT, SUM, AVG, etc.).
  - Use if applicable **different order by methods** (ABS, ASC, DESC, etc.) if not mentioned vaguely in the prompt."""

  prompt += f"""
  - **If the query can be rewritten using a different table as the base table, do so.**
  - **Test each query using query_sql_database_tool tool if incorrect, rewrite the query.**
  - **If not explicitly said, also experiment with or without using DISTINCT**.
  - Tools are for internal use to gather information before writing SQL — not part of the SQL query itself.
  - **When the question asks for the "most" or "least" amount paid, or any similar question related to totals or overall sums, ensure that you use the SUM() aggregate function and a GROUP BY clause to calculate the total amount for each customer, entity or group.**

  ### **Tools Available**
  Use the following tools as needed:
  - `tool_list_tables`: Lists table names in the database.
  - `tool_info_schema`: Shows table schema and sample rows.
  - `tool_run_query`: Executes SQL queries to get results. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use `info_sql_database_tool` to query the correct table fields

  ## ❗ Do NOT misuse tools inside SQL queries
  🚫 NEVER include tool names (e.g., `tool_list_tables`, `tool_info_schema`, `tool_run_query`) inside a SQL statement.
  - They are NOT SQL functions.
  - NEVER write `SELECT tool_list_tables()` or anything similar.
  - These tools are **used before writing SQL** to gather information about the database structure.

  ## External Knowledge
  - Bonding/bonded also means connected
  - When asked about an object that doesn't have an exact column about it (such as "atoms") try to use the ID column of it (like column atom_id from atom or atom_id from connected or atom_id2 from connected)
  - When asked find object x from y which has object z, also mention object y in the SELECT column.
  - When you hesitate between objects to SELECT, use all 2 or 3. Rather have too much information than less.
  - Given a question/prompt with "between" place it in the query with the appropriate values. Like: Prompt:Among the molecules between TR004 to TR010, how many of them has single bonds? Answer: SELECT COUNT(DISTINCT T.molecule_id) FROM bond AS T WHERE T.molecule_id BETWEEN 'TR004' AND 'TR010' AND T.bond_type = '-'
  - When given answers with a lot of NONE values, try to make a query that says to show no "NONE" or aren't null

  - Before writing any SQL, ALWAYS first:
    1. Use `tool_list_tables` to get the list of available tables.
    2. Then use `tool_info_schema` on relevant tables to view columns and relationships.
    3. Only after gathering table and schema info, generate valid SQL using actual table/column names.

  ### **Response Format**
  """

  if query_amount == 1:
    prompt += f"""Your response **must only contain the final SQL query**, without any explanation or reasoning."""
  else:
    prompt += f"""Your response **must only contain the final SQL queries**, without any explanation or reasoning."""

  prompt += f"""
    ✅ Valid SQL starts with SELECT, INSERT, UPDATE, etc.
    🚫 Invalid SQL includes non-SQL tool names like `tool_list_tables()`.
    """
  # databasellm_prompt = PromptTemplate(template=prompt, input_variables=["input"])
  messages = [prompt, HumanMessagePromptTemplate.from_template("{input}"), AIMessage(content=SQL_FUNCTIONS_SUFFIX), MessagesPlaceholder(variable_name="agent_scratchpad")]

  databasellm_prompt = ChatPromptTemplate.from_messages(messages)

  runnable = create_tool_calling_agent(databasellm, databasellm_tools, databasellm_prompt)  # type: ignore

  agent = RunnableMultiActionAgent(runnable=runnable,input_keys_arg=["input"], return_keys_arg=["output"])

  databasellm_agent = AgentExecutor(
          name="SQL Agent Executor",
          agent=agent,
          tools=databasellm_tools,
          verbose=False,
          max_iterations=40,
          max_execution_time=None,


      )
  print("Database Agent created!")
  return databasellm_agent, list_sql_database_tool, info_sql_database_tool

#### Configure Manager Agent

In [ ]:
def limiting_queries(query):
    if "LIMIT" not in query.upper() and "COUNT" not in query.upper():
        return query + " LIMIT 8"
    else:
        return query

In [ ]:
def limiting_queries(query: str) -> str:
    """Applies LIMIT only if it's a non-aggregate SELECT query."""
    import re
    is_aggregate = bool(re.search(r'\b(COUNT|SUM|AVG|MIN|MAX)\b', query, re.IGNORECASE))
    if "SELECT" in query.upper() and not is_aggregate and "LIMIT" not in query.upper():
        optimized_query = query.rstrip(";") + " LIMIT 50;"
    else:
        optimized_query = query
    return optimized_query

def extract_tables_from_queries(queries: List[str]) -> List[str]:
    """Extracts table names from a list of SQL queries."""
    import re
    tables = set()
    for query in queries:
        found_tables = re.findall(r"\bFROM\s+([\w_]+)", query, re.IGNORECASE)
        tables.update(found_tables)
        found_joins = re.findall(r"\bJOIN\s+([\w_]+)", query, re.IGNORECASE)
        tables.update(found_joins)
    return list(tables)

def is_suspicious_result(query, result):
    # Flatten the result
    flat = [item for row in result for item in row]

    # Case: percentage query returns 0 or None
    if "percentage" in query.lower() or "percent" in query.lower():
        if all(val in [None, 0, 0.0] for val in flat):
            return True

    # Case: ID list query returns only nulls
    if "id" in query.lower() and all(val in [None, ""] for val in flat):
        return True

    return False

def is_valid_result(result):
    try:
        # Flatten nested tuples like [('0.66',)] to just values
        flat = [item for row in result for item in row]
        for val in flat:
            if isinstance(val, (int, float)) and val > 0:
                return True
            if isinstance(val, str) and val.strip() not in ('0', '', 'null', 'None'):
                return True
    except Exception:
        pass
    return False

In [ ]:
from collections import defaultdict
def create_manager_agentv3(databasellm_agent, seniorllm_agent, refinerllm_agent, list_sql_database_tool, info_sql_database_tool):
  # Store errors per session
  error_history_store = defaultdict(list)

  @tool
  def generate_sql_query_tool(prompt: str) -> list:
      """
      This tool generates SQL queries based on a user's natural language input.
      Optionally it can invoke the seniorllm_agent for more information.
      Afterwards each query is executed in the execute_sql_query_tool function.
      Optionally within the execute_sql_query_tool function, the refinerllm_agent can be used to refine the queries.

      Args:
          prompt (str): The user's natural language input.

      Returns:
          list: A list of the results of the SQL queries, which you can use to answer the prompt/question.
      """
      extended_prompt = f"""
      Based on the following question, generate **a single** accurate SQL query given the prompt. **ALWAYS** assume that it is possible to answer the given question with given prompt information with the database.

      **User Question:** {prompt}
      """

      if seniorllm_agent != None:
        senior_response = seniorllm_agent.invoke(prompt)['output']

        # Extract tables
        tables_match = re.search(r"\*\*Tables:\*\* `(.*?)`", senior_response, re.DOTALL)
        tables = tables_match.group(1).split(", ") if tables_match else []

        # Extract relevant columns per table
        columns_dict = {}
        column_matches = re.findall(r"- `(\w+)` → (.*?)\n", senior_response)

        for table, columns in column_matches:
            column_list = [col.strip() for col in columns.split(",")]
            columns_dict[table] = column_list

        # Extract explanation
        explanation_match = re.search(r"\*\*Explanation of relevance:\*\*\n(.*)", senior_response, re.DOTALL)
        explanation = explanation_match.group(1).strip() if explanation_match else ""

        structured_response = {
            "tables": tables,
            "columns": columns_dict,
            "explanation": explanation}

        relevant_info_str = str(structured_response) if structured_response else "No relevant info found."
        extended_prompt += f"""
      **Relevant Info:** {relevant_info_str}
      """
      extended_prompt += f"""
      - Use only the tables and columns listed as relevant.
      - Include all user-requested fields.
      - Use JOINs when combining data from multiple tables.
      - Add filters (WHERE, LIKE, IN) if needed to fulfill the user's question.
      - Use aggregation functions (COUNT, AVG, SUM) when necessary.
      - If superlatives are mentioned (e.g., "highest"), use ORDER BY and LIMIT.
      - Format the query with proper syntax.
      - **Return only the SQL query.**
      - Only use tables that are mentioned as relevant info OR in the info sql database tool.
      - Also fluctuate the where clause generation of each query, by using "LIKE '%word%'" instead of "= 'thisisalongword'.
      - Also fluctuate the generation of each query, for between tables and without or without GROUP BY
      - Use **different aggregation methods** (COUNT, SUM, AVG, etc.).
      - Use if applicable **different order by methods** (ABS, ASC, DESC, etc.) if not mentioned vaguely in the prompt.
      - Ensure syntax correctness before returning.
      - **Return only the final SQL query**, without explanations.
      - **Pay close attention to all requested columns in the user question.**
      - **Use joins to combine data from multiple tables when necessary.**
      - **Include all requested information in the query result, including school type, school name, and latitude.**
      - **Use the correct table aliases (T1, T2, etc.) to avoid ambiguity when joining tables.**
      - **If the question requests the "highest" or "lowest" value, use ORDER BY and LIMIT 1 to find the result.**
      - **Use the correct column names from the tables, if the column names are slightly different from the question, use the column names from the tables.**
      - **If the question asks for a specific "type" or "category", make sure to include the corresponding column in the SELECT statement.**
      - **Return only the final SQL query**, without explanations.
      """
      response = databasellm_agent.invoke(extended_prompt)
      sql_blocks = re.findall(r"```sql\s*([\s\S]*?)\s*```", response['output'], re.MULTILINE)
      query_in_list = [re.sub(r'\s+', ' ', sql.strip()) for sql in sql_blocks]

      global sql_query_store
      sql_query_store.append(query_in_list)
      # Validate by running the query with LIMIT 1
      try:
          db.run(f"{query_in_list[0]} LIMIT 1")
          status = "GOOD"
      except Exception as e:
          status = "NOT GOOD: " + str(e)
      good_results = execute_sql_query(extended_prompt, sql_query_store)
      return good_results

  manager_tools = [generate_sql_query_tool]

  def execute_sql_query(extended_prompt: str, sql_query_store: list) -> list:
    """Executes SQL queries, separates good and bad, and uses refiner agent up to 3 times if needed."""
    good_queries = []
    good_results = []

    bad_queries = []
    bad_results = []

    global db
    global refinerllm_agent

    # print("sql_query_store", sql_query_store)

    # 1. First pass: Try all current queries
    for query in sql_query_store[-1]:
        optimized_query = limiting_queries(query)
        try:
            result = db.run(optimized_query)

            # ✅ Treat empty or suspicious results as failure
            if not result or is_suspicious_result(query, result):
                print(f"Query returned suspicious result: {result} — retry or refine")
                raise Exception(f"Query returned suspicious result: {result} — retry or refine")

            # ✅ If we get a meaningful result, break the loop
            if is_valid_result(result):  # <-- NEW check
                print(f"✅ Query returned valid result: {result}")
                good_queries.append(optimized_query)
                good_results.append(result)
                break  # ✅ STOP HERE

            # Optional fallback if you're unsure about floats
            print(f"⚠️ Query returned a non-empty but unconfirmed result: {result}")
            raise Exception("Unconfirmed result content")

        except Exception as e:
            print(f"❌ Query returned incorrect: {e}")
            bad_queries.append(optimized_query)
            bad_results.append(f"{str(e)} — refine")

    attempts = 3
    # 2. If all failed, retry using the refiner agent (up to 3 times)
    if not good_queries and refinerllm_agent is not None:
        print("REFINER USED!!")
        for attempt in range(attempts):
            # print(f"\n--- Refinement Attempt #{attempt + 1} ---")
            refine_input = extended_prompt

            refine_input += "\n**Incorrect Queries:**"
            for i, query in enumerate(bad_queries):
                refine_input += f"\n{i+1}: {query}"

            refine_input += "\n**Incorrect Results:**"
            for i, result in enumerate(bad_results):
                refine_input += f"\n{i+1}: {result}"

            refine_input += """
- The query must be **structurally unique** (different FROM tables, joins, or filters).
- Only use tables that are mentioned as relevant info OR in the info sql database tool.
- **Analyze the error messages to identify the cause of the failures.**
- **Use the schema information to ensure the corrected queries are valid.**
- **Make minimal changes to the original queries to fix the errors.**
- **Ensure the corrected query is valid against the database schema.**
- **Avoid the error messages by using different tables and columns that caused the error messages.**
- **Don't make the same query as either the good or bad queries.**
- **Return only the final SQL query**, without explanations.
"""

            # print("Invoking refiner with input:", refine_input)
            response = refinerllm_agent.invoke(refine_input)

            # Extract SQL from response
            # print("Refiner response:", response['output'])
            sql_blocks = re.findall(r"```sql\s*([\s\S]*?)\s*```", response['output'], re.MULTILINE)
            query_in_list = [re.sub(r'\s+', ' ', sql.strip()) for sql in sql_blocks]

            if not query_in_list:
                print("No SQL query found in refiner output.")
                continue

            extracted_sql_query = query_in_list[0]
            optimized_refined_query = limiting_queries(extracted_sql_query)

            try:
                result = db.run(optimized_refined_query)
                if result == '':
                    raise Exception("Refined query returned empty result.")
                sql_query_store[-1].append(optimized_refined_query)
                good_results.append(result)

                print("Refined query succeeded.: ", sql_query_store[-1])
                break  # Exit loop if successful
            except Exception as e:
                print(f"Refined query attempt #{attempt+1} failed: {e}")
                bad_queries.append(optimized_refined_query)
                bad_results.append(str(e))
        return good_results
    return good_results

  prompt = """
  You are a SQL Manager Agent. Your job is to:
  - Generate a valid SQL query in SQLite using the tool `generate_sql_query_tool`.
  - Summarize the results from that query based on the user's question.

  ---

  ## 🧠 STRICT RULES:

  1. **ALWAYS call `generate_sql_query_tool` ONCE and ONLY ONCE.**
  2. **NEVER call it again**, no matter the output.
  3. The function `execute_sql_query` will handle validation. Your job is NOT to recheck or re-call the query.
  4. You MUST stop after the first SQL tool call, even if:
    - It returns an empty list
    - It returns a failed or unexpected output
    - You are unsure
  5. ONLY return the final answer based on the result.

  ---

  ## 🔎 Output Format:
  - First, use `generate_sql_query_tool` ONCE.
  - Then respond with a short, natural language answer using the results **you received** (assume it's valid unless explicitly told otherwise).
  - NEVER call the tool again.

  ---

  ## ✅ Example:

  User: "How many employees work in each department?"

  You:
  1. Call: `generate_sql_query_tool("SELECT department, COUNT(*) FROM employees GROUP BY department;")`
  2. Result: `[("Sales", 45), ("HR", 10)]`
  3. Answer: `"The Sales department has the most employees (45), while HR has the least (10)."`

  ---

  Remember:
  - If the result is empty, say "No results were found."
  - If the result is invalid, say "There was an error with the query."
  - You NEVER fix or retry the SQL query. Just report.
  """


  messages = [
      prompt,
      HumanMessagePromptTemplate.from_template("User Query: {input}"),
      AIMessage(content=SQL_FUNCTIONS_SUFFIX),
      MessagesPlaceholder(variable_name="agent_scratchpad")
  ]

  manager_prompt = ChatPromptTemplate.from_messages(messages)

  runnable = create_tool_calling_agent(managerllm, manager_tools, manager_prompt)  # type: ignore

  agent = RunnableMultiActionAgent(
      runnable=runnable,
      input_keys_arg=["input"],
      return_keys_arg=["output"]
  )
  manager_agent = AgentExecutor(
      name="Manager Agent Executor",
      agent=agent,
      tools=manager_tools,
      verbose=True,
      max_iterations=None,
      max_execution_time=None,
  )
  print("Manager Agent created!")
  return manager_agent

#### Configure Seniorllm Agent

In [ ]:
def configure_seniorllm():
    @tool
    def get_tables_columns_samples() -> str:
        """
        Retrieves all available tables and their columns, ensuring all tables that could
        be remotely related to the question are included.
        """
        # print('3. Analyzing database schema...')
        try:
            table_names = ", ".join(db.get_usable_table_names())
            table_info = db.get_table_info([t.strip() for t in table_names.split(",")])
            return table_info
        except Exception as e:
            return f"Error retrieving database schema: {str(e)}"

    seniorllm_tools = [get_tables_columns_samples]

    prompt = f"""
    You are a **Senior AI Database Analyst** with expert SQL knowledge.
    Your role is to **identify ALL relevant tables and columns** that could possible help answer the question AND potentially try to explain any abbrevations.

    ### **STRICT GUIDELINES**
    - **MANDATORY**: Use available tools to retrieve the database schema.
    - **NEVER** assume or invent tables/columns.
    - **DO NOT** return just table names—always include at least one relevant column per table.
    - **ALWAYS** return every table that could possibly contribute to the answer, even if the relationship is indirect.
    - **CROSS-REFERENCE** related tables (e.g., for sales: include `orders`, `orderdetails`, `payments`, not just `products`).
    - **IF UNCERTAIN**, err on the side of including rather than excluding a table.

    ### **HOW TO WORK**
    1. Use `get_tables_columns_samples` to retrieve the full database schema.
    2. Find and identify any possible related table(s) directly related to the question.
    4. Provide an **explanation** of why each table is relevant.

    ### **RESPONSE FORMAT**
    - **Table → Relevant Columns**
    - **Explanation of relevance**

    ✅ Example:
    **Tables:** `orders`, `orderdetails`, `payments`, `products`
    **Relevant Columns:**
    - `orderdetails` → `product_id`, `quantity` (Needed to track sold products)
    - `orders` → `order_id`, `order_date` (Provides context on when sales happened)
    - `payments` → `amount`, `payment_date` (Links revenue to sales)
    - `products` → `product_id`, `expected_profit` (Identifies profitable products)
    - `frpm` → `CDSCode` (The Free or Reduced-Price Meals table has a column named CDSCode which helps to identify the schools based on the reduced-price meals)
    """
    messages = [
        prompt,
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]

    seniorllm_prompt = ChatPromptTemplate.from_messages(messages)

    runnable = create_tool_calling_agent(
        seniorllm, seniorllm_tools, seniorllm_prompt
    )  # type: ignore

    agent = RunnableMultiActionAgent(
        runnable=runnable, input_keys_arg=["input"], return_keys_arg=["output"]
    )

    seniorllm_agent = AgentExecutor(
        name="SQL Agent Executor",
        agent=agent,
        tools=seniorllm_tools,
        verbose=False,
        max_iterations=5,
        max_execution_time=60,
    )
    print("Senior Agent created!")
    return seniorllm_agent


#### Configure refinerllm

In [ ]:
def configure_refinerllm(db: SQLDatabase, database_name: str, bucket_name: str, few_shot_path: str, refinerllm: Union[ChatOpenAI, ChatVertexAI], query_table_name='SQL'):
  list_sql_database_tool = ListSQLDatabaseTool(db=db)

  info_sql_database_tool_description = (
      "Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. "
      f"Be sure that the tables actually exist by calling {list_sql_database_tool.name} first! "
      "Example Input: table1, table2, table3"
  )
  info_sql_database_tool = InfoSQLDatabaseTool(db=db, description=info_sql_database_tool_description)

  query_sql_database_tool_description = (
      "Input to this tool is a detailed and correct SQL query, output is a "
      "result from the database. If the query is not correct, an error message "
      "will be returned. If an error is returned, rewrite the query, check the "
      "query, and try again. If you encounter an issue with Unknown column "
      f"'xxxx' in 'field list', use {info_sql_database_tool.name} "
      "to query the correct table fields."
  )
  query_sql_database_tool = QuerySQLDataBaseTool(db=db, description=query_sql_database_tool_description)

  refinerllm_tools = [
      list_sql_database_tool,
      info_sql_database_tool,
      query_sql_database_tool,
  ]

  # Load training data from GCS
  training_data = load_json_from_gcs(bucket_name, few_shot_path)

  database_train = [entry for entry in training_data if entry.get("db_id") == database_name]
  few_shot_examples = "\n".join(
      [f"Question: {ex['question']}\nSQL: {ex[query_table_name]}" for ex in database_train[:5]]
  )

  prompt = f"""
  You are an AI SQL agent specializing in generating and executing **syntactically correct SQL queries** in SQLite language.
  Return a single final SQL query—without explanations, reasoning, or extra text—that can be valid answers to the prompt.

  ### **Database Knowledge**
  Examples of correct queries based on the database structure:
  {few_shot_examples}

  ### **SQL Generation Rules**
  - Before including any table in a SQL query, confirm its existence using `list_sql_database_tool`.
  - If `query_sql_database_tool` returns an error indicating a table or column does not exist, use `info_sql_database_tool` to get the correct names and rewrite the query.
  - When generating a query, follow these steps:
      1. Use `list_sql_database_tool` to verify table existence.
      2. If needed, use `info_sql_database_tool` to get table schema.
      3. **Analyze the error messages from previous attempts to identify the specific tables and columns that caused errors. Avoid using these tables and columns in subsequent queries.**
      4. Generate the SQL query using the error message information, and the schema information.
      5. Use `query_sql_database_tool` to validate the query. If an error occurs, return to step 2.

  ### **Tools Available**
  Use the following tools as needed:
  - `info_sql_database_tool`: Lists table names in the database.
  - `query_sql_database_tool`: Execute SQL queries to valide. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use `info_sql_database_tool` to query the correct table fields

  ### Abbreviation Handling
  When you encounter abbreviations, follow these steps:
  1.  Try to infer their full meaning from the surrounding context and the database schema.
  2.  Pay close attention to column names and data types.
  3.  Use the `info_sql_database_tool` to examine column data if needed.
  4.  If unsure, reason about possible meanings based on the database context.

  Example: If the question is 'Find schools with CDSCode 12345' and the database has a 'schools' table with a 'CDSCode' column, the SQL query should be 'SELECT * FROM schools WHERE CDSCode = '12345';' because CDSCode is a school identifier.

  ### **Refinement Process**
  You will be provided with a list of bad SQL queries that failed to execute, along with their corresponding error messages in a dictionary. Your task is to analyze these errors and generate corrected SQL queries that don't have tables and columns that cause errors as provided in the error messages.

  ### **Error Analysis**
  Common SQL errors include:
  - "unknown column": The column name is incorrect.
  - "syntax error": The SQL syntax is invalid.
  - "table not found": The table name is incorrect.
  - "ambiguous column name": a column name is used that exist in multiple tables, and the table is not specified.

  ### **Input Format**
  You will a prompt that will indicate the question/prompt, the failed queries (and which tables were incorrect used) and the correct queries.

  ### **Refinement Rules**
  - **Thoroughly analyze the error messages to identify the specific tables and columns that caused the errors.**
  - **Avoid using tables and columns that caused previous errors.**
  - If the results contain lots of NONE values, try to find the column that caused it and make a WHERE clause that says "is not null" like "CharterNum is not null"
  - Use the `info_sql_database_tool` to get the correct schema information.
  - **Make significant changes to the original query to fix the error, avoiding the use of the tables and columns that caused previous errors.**
  - **Ensure the corrected query is significantly different from all previous attempts.**
  - Ensure the corrected query is valid against the database schema.
  - Avoid using Where clauses with exactly the correct phrase, rather use "LIKE '%word%'" instead of  "= 'word'" and look at the possible values in the column rather than the question. Example prompt: How many unified schools? -> LIKE '%unified school%'

  ### **Response Format**
  Your response **must only contain the final SQL query string**, without any explanation or reasoning.
  """

  # databasellm_prompt = PromptTemplate(template=prompt, input_variables=["input"])
  messages = [prompt, HumanMessagePromptTemplate.from_template("{input}"), AIMessage(content=SQL_FUNCTIONS_SUFFIX), MessagesPlaceholder(variable_name="agent_scratchpad")]

  refinerllm_prompt = ChatPromptTemplate.from_messages(messages)

  runnable = create_tool_calling_agent(refinerllm, refinerllm_tools, refinerllm_prompt)  # type: ignore

  agent = RunnableMultiActionAgent(runnable=runnable,input_keys_arg=["input"], return_keys_arg=["output"])

  refinerllm_agent = AgentExecutor(
          name="Refiner Agent Executor",
          agent=agent,
          tools=refinerllm_tools,
          verbose=False,
          max_iterations=20,
          max_execution_time=20,
      )
  print("Refiner Agent created!")
  return refinerllm_agent

#### Extracting Training Equipment

In [ ]:
def compute_sql_complexity(sql_query):
    """Computes complexity based on QSC, Parse Tree Depth, and Token Length."""
    parsed = sqlparse.parse(sql_query)

    # 1. Query Token Length
    token_length = len([t for t in parsed[0].tokens if t.value.strip()])

    # 2. Query Structural Complexity (based on SQL keywords)
    join_count = sql_query.upper().count("JOIN")
    agg_count = sum(sql_query.upper().count(kw) for kw in ["COUNT", "SUM", "AVG", "MIN", "MAX"])
    filter_count = sql_query.upper().count("WHERE")
    order_count = sql_query.upper().count("ORDER BY")
    group_count = sql_query.upper().count("GROUP BY")
    having_count = sql_query.upper().count("HAVING")

    qsc_score = (2 * join_count) + (2 * agg_count) + filter_count + order_count + group_count + having_count

    # 3. Parse Tree Depth (approximated by nested parentheses)
    tree_depth = sql_query.count("(")

    return [qsc_score, tree_depth,token_length]

In [ ]:
def benchmarkingOnBirdBench(test_example, databasellm_agent, managerllm_agent, seniorllm_agent, refinerllm_agent, query_table_name, use_manager_agent=False, use_senior_agent=False, use_refiner_agent=False):
  session_id = str(uuid.uuid4())
  responses = []  # List to store all response data
  last_sql_query_collection = None
  for index, example in enumerate(test_example):
      db_id = example["db_id"]
      question = example["question"]
      correct_sql = example[query_table_name]
      try:
        difficulty = example["difficulty"]
      except:
        difficulty = "medium"
      qsc_score, tree_depth, token_length = compute_sql_complexity(correct_sql)
      # Create a dictionary to store response details
      response_data = {
          "database_name": db_id,
          "question_number": index+1,
          "question": question,
          "correct_sql": correct_sql,
          "correct_answer": None,
          "generated_output": None,
          "generated_direct_output": None,
          "response_status": None,
          "iterations": None, #optionally a way to calculate how many iterations the LLM made? To know the thinking process a bit
          "qsc_score": qsc_score,
          "parse_tree_depth": tree_depth,
          "token_length": token_length,
          "difficulty": difficulty,
          "time_spent": None,
          "generated_query_runtime": [],
          "correct_query_runtime": None,
      }
      # Track start time for latency calculation & query storage
      start_time = time.time()

      # Execute the agent and get the response
      print("Response calculating...")
      time_spent_start = time.time()
      if not use_manager_agent: #using only database_agent
        database_agent_prompt = f"""
        Based on the following question, generate **a single** accurate SQL query given the prompt. **ALWAYS** assume that it is possible to answer the given question with given prompt information with the database.

        **User Question:** {question}
        - Only use tables that are mentioned as relevant info OR in the info sql database tool.
        - Also fluctuate the where clause generation of each query, by using "LIKE '%word%'" instead of "= 'thisisalongword'.
        - Also fluctuate the generation of each query, for between tables and without or without GROUP BY
        - Use **different aggregation methods** (COUNT, SUM, AVG, etc.).
        - Use if applicable **different order by methods** (ABS, ASC, DESC, etc.) if not mentioned vaguely in the prompt.
        - Ensure syntax correctness before returning.
        - **Return only the final SQL query**, without explanations.
        - **Pay close attention to all requested columns in the user question.**
        - **Use joins to combine data from multiple tables when necessary.**
        - **Include all requested information in the query result, including school type, school name, and latitude.**
        - **Use the correct table aliases (T1, T2, etc.) to avoid ambiguity when joining tables.**
        - **If the question requests the "highest" or "lowest" value, use ORDER BY and LIMIT 1 to find the result.**
        - **Use the correct column names from the tables, if the column names are slightly different from the question, use the column names from the tables.**
        - **If the question asks for a specific "type" or "category", make sure to include the corresponding column in the SELECT statement.**
        - **Return only the final SQL query**, without explanations.
        """
        response = databasellm_agent.invoke(database_agent_prompt)
        sql_blocks = re.findall(r"```sql\s*([\s\S]*?)\s*```", response['output'], re.MULTILINE)
        # Clean and format each SQL query
        query_in_list = [re.sub(r'\s+', ' ', sql.strip()) for sql in sql_blocks]
      else:
        response = managerllm_agent.invoke(question)
        global sql_query_store
        query_in_list = sql_query_store[-1]

      response_data["time_spent"] = time.time() - time_spent_start

      generated_direct_outputs = []
      for query in query_in_list:
        try:
          generated_query_start_time = time.time()
          generated_direct_output = db.run(r"{}".format(query))
          response_data["generated_query_runtime"].append(time.time() - generated_query_start_time)
        except Exception as e:
          generated_direct_output = str(e)
          response_data["generated_query_runtime"].append(None)
        generated_direct_outputs.append(generated_direct_output)

      try:
        correct_sql_start_time = time.time()
        correct_answer = db.run(r"{}".format(correct_sql))
        response_data["correct_query_runtime"] = time.time() - correct_sql_start_time
      except:
        correct_answer = "ANSWER IS INCORRECT: INVALID QUESTION"
        response_data["correct_query_runtime"] = None

      # Update response data
      response_data.update({
          "generated_output": response['output'], #change this to the response of toolcaller agent if enabled
          "generated_sqls": query_in_list,
          "generated_direct_output": [generated_direct_outputs], #change this for multi agents
          "correct_answer": correct_answer,
          "response_status": "Success",
      })

      # Debugging output
      print(f"🔹  Question {index+1}: {question}")
      print(f"> Generated Output: {response['output']}")
      print(f"> Correct Output: {correct_answer}")
      print(f"> Correct sql_query: {correct_sql}")
      print(f"> Generated SQLs: {query_in_list}")
      print(f"> Generated Direct Output: {generated_direct_outputs}")
      # print(f"> Tokens Used: {response_data['tokens_used']}")
      print(f"> Qsc_score: {qsc_score}\n")
      responses.append(response_data)
      print(f"NEXT QUESTION!")
  return responses

#### Benchmarking Loop


##### Loading functions

In [ ]:
def load_json_from_gcs(bucket_name, blob_path):
    """Loads a JSON file from Google Cloud Storage."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_path)

    try:
        json_data = json.loads(blob.download_as_text())
        return json_data
    except Exception as e:
        print(f"Error loading JSON from GCS: {e}")
        return None

In [ ]:
def upload_json_to_gcs(bucket_name, blob_path, file_name, data):
    """Uploads JSON data to Google Cloud Storage."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob_path = os.path.join(output_folder, file_name)
    blob = bucket.blob(blob_path)

    try:
        json_string = json.dumps(data, indent=4)
        blob.upload_from_string(json_string, content_type="application/json")
        print(f"JSON data uploaded to gs://{bucket_name}/{blob_path}")
    except Exception as e:
        print(f"Error uploading JSON to GCS: {e}")

##### Running

In [ ]:
# Store errors per session
error_history_store = defaultdict(list)
database_names = test_databases_names
databases = test_databases
dataset_folder = test_dataset_folder
json_data = load_json_from_gcs(bucket_name, question_path)
sql_query_store = []
for database_name in database_names[1:]:
  custom_prompts = [item for item in json_data if item.get('db_id') == database_name]
  if len(custom_prompts) != 0:
    print(f"""Database "{database_name}" contains {len(custom_prompts)} prompts""")
    db = databases[database_name]

    databasellm_agent, list_sql_database_tool, info_sql_database_tool = configure_databasellmv2(database_name, bucket_name, few_shot_path, few_shot_amount, query_amount, query_table_name)
    if use_manager_agent:
      if use_senior_agent:
        seniorllm_agent = configure_seniorllm()
      else:
        seniorllm_agent = None
      if use_refiner_agent:
        refinerllm_agent = configure_refinerllm(db, database_name, bucket_name, few_shot_path, refinerllm, query_table_name)
      else:
        refinerllm_agent = None
      managerllm_agent = create_manager_agentv3(databasellm_agent, seniorllm_agent, refinerllm_agent, list_sql_database_tool, info_sql_database_tool)
    else:
      managerllm_agent = None

    responses = benchmarkingOnBirdBench(custom_prompts, databasellm_agent, managerllm_agent, seniorllm_agent, refinerllm_agent, query_table_name, use_manager_agent, use_senior_agent, use_refiner_agent)

    output_file = f"{database_name}_responses.json"
    with open(output_file, "w", encoding="utf-8") as f:
      json.dump(responses, f, indent=4)

    blob_path = os.path.join(output_folder, output_file)

    upload_json_to_gcs(bucket_name, blob_path, output_file, responses)

    # drive_output_file = "/content/drive/MyDrive/Colab/benchmarks/bird-bench/geminiv5/" + output_file
    # with open(drive_output_file, "w", encoding="utf-8") as f:
    #   json.dump(responses, f, indent=4)

    print(f"""Database "{database_name}" completed\n""")
  else:
    print(f"""Database "{database_name}" has no prompts\n""")